In [74]:
import pandas as pd

In [75]:
items = [
    {"type": "drilled_hole", "from": 998.8, "to": 1100, "diam": 149, },
    {"type": "drilled_hole", "from": 0, "to": 12, "diam": 438, "label": ""},
    {"type": "drilled_hole", "from": 186, "to": 998.8, "diam": 222, "label": "main hole"},
    {"type": "drilled_hole", "from": 12, "to": 186, "diam": 311},
    {"type": "casing", "from": 0, "to": 12, "inner_diam": 326, "label": "conductor"},
    {"type": "casing", "from": 0, "to": 996.5, "inner_diam": 162, "label": "7\" API casing"},
    {"type": "casing", "from": 0, "to": 184, "inner_diam": 227, "label": "artesian control"},
    {"type": "casing", "from": 0, "to": 320, "inner_diam": 104, "label": "fictional reline"}
]

__version__ = 'dev'

In [76]:
def sort_items(items, schema_type='drilled_hole_casing_pzone'):
    if schema_type == 'drilled_hole_casing_pzone':
        return sort_items_under_schema_drilled_hole_casing_pzone(items)
    else:
        raise KeyError(f'schema_type {schema_type} is not recognised by well_schematics v{__version__}')
        
def sort_items_under_schema_drilled_hole_casing_pzone(items):
    sorted_items = {}
    for item_type in ('drilled_hole', 'casing'):
        diam_key = {'drilled_hole': "diam", "casing": "inner_diam"}[item_type]
        sub_items = [x for x in items if x['type'] == item_type]
        idxs = [x for x in range(len(sub_items))]
        order_depths = sorted(idxs, key=lambda x: (sub_items[x]['from'], sub_items[x]['to']))
        order_diams = sorted(idxs, key=lambda x: sub_items[x][diam_key])
        sub_items = tuple([
            dict(
                i=i, 
                order_depth=order_depths.index(i), 
                order_diam=order_diams.index(i), 
                **sub_items[i]
            ) for i in idxs
        ])
        sorted_items[item_type] = sub_items
    return sorted_items

In [77]:
pd.DataFrame(sort_items(items)["drilled_hole"]).sort_values('order_depth')

,i,order_depth,order_diam,type,from,to,diam,label
1,1,0,3,drilled_hole,0.0,12.0,438,
3,3,1,2,drilled_hole,12.0,186.0,311,NaN
2,2,2,1,drilled_hole,186.0,998.8,222,main hole
0,0,3,0,drilled_hole,998.8,1100.0,149,NaN


In [78]:
pd.DataFrame(sort_items(items)["drilled_hole"]).sort_values('order_diam')

,i,order_depth,order_diam,type,from,to,diam,label
0,0,3,0,drilled_hole,998.8,1100.0,149,NaN
2,2,2,1,drilled_hole,186.0,998.8,222,main hole
3,3,1,2,drilled_hole,12.0,186.0,311,NaN
1,1,0,3,drilled_hole,0.0,12.0,438,


In [79]:
pd.DataFrame(sort_items(items)["casing"]).sort_values('order_depth')

,i,order_depth,order_diam,type,from,to,inner_diam,label
0,0,0,3,casing,0,12.0,326,conductor
2,2,1,2,casing,0,184.0,227,artesian control
3,3,2,0,casing,0,320.0,104,fictional reline
1,1,3,1,casing,0,996.5,162,"7"" API casing"


In [80]:
pd.DataFrame(sort_items(items)["casing"]).sort_values('order_diam')

,i,order_depth,order_diam,type,from,to,inner_diam,label
3,3,2,0,casing,0,320.0,104,fictional reline
1,1,3,1,casing,0,996.5,162,"7"" API casing"
2,2,1,2,casing,0,184.0,227,artesian control
0,0,0,3,casing,0,12.0,326,conductor
